### 역정보추가크롤링

In [1]:
# 패키지 임포트
import selenium
from selenium import webdriver
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm_notebook
import urllib
from urllib.request import urlopen
import requests

In [182]:
gu = ['강남구','강동구', '강북구', '강서구', '관악구', '광진구', '구로구', '금천구', '노원구', '도봉구', '동대문구', '동작구', '마포구', '서대문구', '서초구', '성동구', '성북구','은평구' ,'송파구']

In [184]:
for g in tqdm_notebook(gu):
    df = pd.read_csv('./' + g +'_맛집.csv', encoding='CP949')
    base_url='https://map.naver.com/v5/?c=14127676.3957465,4523397.9832608,15,0,0,0,dh'
    driver = webdriver.Chrome('./driver/chromedriver')
    driver.get(base_url)

    url_list = []
    for res in df['업소명'] :
        try :
            search_xpath = '/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-input-box/div/div[1]/div/input'
            element = driver.find_element(By.XPATH, search_xpath)
            time.sleep(0.5)
            element.clear()
            element.send_keys(res)
            element.send_keys("\n")
            time.sleep(3)

            url = driver.current_url
            if(url.split('/').__contains__('place')) :
                u = url.split('/')[-1].split('?')[0]
                url_list.append(u)
            else :
                driver.switch_to.frame('searchIframe')
                time.sleep(1)

                target_xpath = '/html/body/div[3]/div/div/div[1]/ul/li[1]/div[2]/a[1]/div/div/span[1]'
                config = driver.find_elements(By.XPATH, target_xpath)
                if len(config) == 0 :
                    target_xpath = '/html/body/div[3]/div/div/div[1]/ul/li[1]/div[1]/a[1]/div/div/span[1]'
                else :
                    target_xpath = '/html/body/div[3]/div/div/div[1]/ul/li[1]/div[2]/a[1]/div/div/span[1]'

                element = driver.find_element(By.XPATH, target_xpath)
                element.click()
                time.sleep(2)

                url = driver.current_url
                u = url.split('/')[-1].split('?')[0]
                url_list.append(u)
                time.sleep(1)

                driver.switch_to.default_content()
                time.sleep(1)

        except :
            url_list.append('-')
            driver.switch_to.default_content()
            time.sleep(1)

    df['업소id'] = url_list

    st_list = []
    for i in range(len(df)):
        try:
            urlnum = df['업소id'][i]
            url = f'https://pcmap.place.naver.com/restaurant/{urlnum}/home'
            driver.get(url)
            html = driver.page_source
            bs_obj = BeautifulSoup(html,'html.parser')
            a = bs_obj.find('div',{'class':'jyfLw'})
            st = a.find('strong').text
            st_list.append(st)
        except :
            st_list.append('-')
            driver.switch_to.default_content()
            time.sleep(1)

    df = df.drop(['업소id'],axis=1)
    df['STATION_NAME'] = st_list

    df.to_csv('./' + g +'_맛집_역명.csv', encoding='CP949')

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_116772/3751299723.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for g in tqdm_notebook(gu):


  0%|          | 0/19 [00:00<?, ?it/s]

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_116772/3751299723.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./driver/chromedriver')
C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_116772/3751299723.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./driver/chromedriver')
C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_116772/3751299723.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./driver/chromedriver')
C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_116772/3751299723.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./driver/chromedriver')
C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_116772/3751299723.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Ser

KeyError: '업소명'

#### 자치구 개별 추가

In [3]:
df = pd.read_csv('./중구_맛집_역명.csv', encoding='CP949')

In [4]:
df

,Unnamed: 0,NAME,ADDRESS,CATE,SUBCATE,STATION_NAME
0,신당역,신당동떡볶이타운,서울 중구,맛집,기타음식,NaN
1,을지로4가역,우래옥본점,서울 중구 창경궁로 62-29,맛집,한식,NaN
2,신당역,마복림떡볶이,서울 중구 다산로35길 5-0,맛집,간이음식,NaN
3,충무로역,필동면옥,서울 중구 서애로 26-0,맛집,한식,NaN
4,동대문역사문화공원역,오장동함흥냉면,서울 중구 마른내로 108-0,맛집,한식,NaN
...,...,...,...,...,...,...
85,NaN,산수갑산,서울 중구 을지로20길 24-0,맛집,한식,NaN
86,NaN,브레라,서울 중구 다산로 41-0,맛집,외국식,NaN
87,NaN,팔레드신[중식],서울 중구 퇴계로 67-0,맛집,외국식,NaN
88,NaN,땡초곱창,서울 중구 퇴계로87길 54-0,맛집,외국식,NaN


In [5]:
base_url='https://map.naver.com/v5/?c=14127676.3957465,4523397.9832608,15,0,0,0,dh'
driver = webdriver.Chrome('./driver/chromedriver')
driver.get(base_url)

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_45532/1601535763.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./driver/chromedriver')


In [6]:
url_list = []
for res in tqdm_notebook(df['NAME']) :
    try :
        search_xpath = '/html/body/app/layout/div[3]/div[2]/shrinkable-layout/div/app-base/search-input-box/div/div[1]/div/input'
        element = driver.find_element(By.XPATH, search_xpath)
        time.sleep(0.5)
        element.clear()
        element.send_keys(res)
        element.send_keys("\n")
        time.sleep(3)
        
        url = driver.current_url
        if(url.split('/').__contains__('place')) :
            u = url.split('/')[-1].split('?')[0]
            url_list.append(u)
        else :
            driver.switch_to.frame('searchIframe')
            time.sleep(1)
            
            target_xpath = '/html/body/div[3]/div/div/div[1]/ul/li[1]/div[2]/a[1]/div/div/span[1]'
            config = driver.find_elements(By.XPATH, target_xpath)
            if len(config) == 0 :
                target_xpath = '/html/body/div[3]/div/div/div[1]/ul/li[1]/div[1]/a[1]/div/div/span[1]'
            else :
                target_xpath = '/html/body/div[3]/div/div/div[1]/ul/li[1]/div[2]/a[1]/div/div/span[1]'
                
            element = driver.find_element(By.XPATH, target_xpath)
            element.click()
            time.sleep(2)

            url = driver.current_url
            u = url.split('/')[-1].split('?')[0]
            url_list.append(u)
            time.sleep(1)

            driver.switch_to.default_content()
            time.sleep(1)
                
    except :
        print('error',res)
        url_list.append('-')
        driver.switch_to.default_content()
        time.sleep(1)

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_45532/3209941687.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for res in tqdm_notebook(df['NAME']) :


  0%|          | 0/90 [00:00<?, ?it/s]

error 양미옥을지로본점
error 광화문국밥본점
error 남산돈가스거리
error 딘타이펑명동중앙점


In [7]:
df['업소id'] = url_list
df

,Unnamed: 0,NAME,ADDRESS,CATE,SUBCATE,STATION_NAME,업소id
0,신당역,신당동떡볶이타운,서울 중구,맛집,기타음식,NaN,13543837
1,을지로4가역,우래옥본점,서울 중구 창경궁로 62-29,맛집,한식,NaN,11679381
2,신당역,마복림떡볶이,서울 중구 다산로35길 5-0,맛집,간이음식,NaN,38405139
3,충무로역,필동면옥,서울 중구 서애로 26-0,맛집,한식,NaN,11678838
4,동대문역사문화공원역,오장동함흥냉면,서울 중구 마른내로 108-0,맛집,한식,NaN,19866570
...,...,...,...,...,...,...,...
85,NaN,산수갑산,서울 중구 을지로20길 24-0,맛집,한식,NaN,18085955
86,NaN,브레라,서울 중구 다산로 41-0,맛집,외국식,NaN,36513384
87,NaN,팔레드신[중식],서울 중구 퇴계로 67-0,맛집,외국식,NaN,1061905251
88,NaN,땡초곱창,서울 중구 퇴계로87길 54-0,맛집,외국식,NaN,12031284


In [8]:
st_list = []
for i in tqdm_notebook(range(len(df))):
    try:
        urlnum = df['업소id'][i]
        url = f'https://pcmap.place.naver.com/restaurant/{urlnum}/home'
        driver.get(url)
        html = driver.page_source
        bs_obj = BeautifulSoup(html,'html.parser')
        a = bs_obj.find('div',{'class':'jyfLw'})
        st = a.find('strong').text
        st_list.append(st)
    except :
        print('error',i)
        st_list.append('-')
        driver.switch_to.default_content()
        time.sleep(1)

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_45532/337708819.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(df))):


  0%|          | 0/90 [00:00<?, ?it/s]

error 46
error 47
error 54
error 62
error 69


In [9]:
st_list

['신당역',
 '을지로4가역',
 '신당역',
 '충무로역',
 '동대문역사문화공원역',
 '명동역',
 '동대입구역',
 '명동역',
 '시청역',
 '동대문역사문화공원역',
 '동대문역사문화공원역',
 '잠실역',
 '신당역',
 '명동역',
 '약수역',
 '명동역',
 '시청역',
 '시청역',
 '신당역',
 '신당역',
 '신당역',
 '을지로입구역',
 '서대문역',
 '시청역',
 '명동역',
 '을지로3가역',
 '동대문역사문화공원역',
 '을지로입구역',
 '명동역',
 '동대입구역',
 '약수역',
 '회현역',
 '을지로3가역',
 '을지로입구역',
 '동대입구역',
 '약수역',
 '을지로4가역',
 '서대문역',
 '신당역',
 '약수역',
 '명동역',
 '동대문역',
 '명동역',
 '을지로입구역',
 '약수역',
 '충무로역',
 '-',
 '-',
 '을지로입구역',
 '동대문역',
 '시청역',
 '명동역',
 '동대입구역',
 '동대입구역',
 '-',
 '명동역',
 '을지로3가역',
 '광화문역',
 '을지로입구역',
 '서울역',
 '신당역',
 '서울역',
 '-',
 '을지로3가역',
 '동대입구역',
 '광화문역',
 '명동역',
 '을지로3가역',
 '광화문역',
 '-',
 '종로3가역',
 '을지로입구역',
 '시청역',
 '충정로역',
 '회현역',
 '시청역',
 '약수역',
 '동대문역사문화공원역',
 '동대문역사문화공원역',
 '신당역',
 '동대입구역',
 '을지로입구역',
 '을지로3가역',
 '을지로3가역',
 '서대문역',
 '을지로4가역',
 '버티고개역',
 '회현역',
 '신당역',
 '을지로입구역']

In [10]:
df = df.drop(['업소id'],axis=1)
df['STATION_NAME'] = st_list

In [11]:
df.to_csv('./중구_맛집_역명.csv',encoding='cp949')